In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/minhkhoi1412/face_recognition_system.git

In [ ]:
os.chdir("/kaggle/working/face_recognition_system")

In [ ]:
os.makedirs('./models')

In [ ]:
!pwd

In [ ]:
!ls -a

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize, Resize, RandomRotation
import numpy as np
from torch.utils.data import DataLoader
from modules.deep_pixel_wise.Dataset_v2 import PixWiseDataset
from modules.deep_pixel_wise.Model import DeePixBiS
from modules.deep_pixel_wise.Loss import PixWiseBCELoss
from modules.deep_pixel_wise.Metrics import predict, test_accuracy, test_loss
from modules.deep_pixel_wise.Trainer import Trainer

# Train

In [ ]:
train_data = pd.read_csv('/kaggle/input/face-anti-spoofing/train_data_celeb_nuaa_kaggle.csv').reset_index(drop=True)
val_data = pd.read_csv('/kaggle/input/face-anti-spoofing/val_data_celeb_nuaa_kaggle.csv').reset_index(drop=True)

In [ ]:
train_data_shuffled = train_data.sample(frac=1).reset_index(drop=True)
train_data_shuffled.head()

In [ ]:
train_data_shuffled.shape

In [ ]:
# Setting device on GPU if available, else CPU
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    print('Max memory reserved: ', round(torch.cuda.max_memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
model = DeePixBiS()
# model.load_state_dict(torch.load('/content/drive/MyDrive/FPT MSE/Capstone Project/Anti_Spoof_DPW/DeePixBiS.pth'))
loss_fn = PixWiseBCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.0001)
train_tfms = Compose([Resize([224, 224]),
                      RandomHorizontalFlip(),
                      RandomRotation(10),
                      ToTensor(),
                    #   Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                      Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_tfms = Compose([Resize([224, 224]),
                     ToTensor(),
                    #  Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                     Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = PixWiseDataset(train_data_shuffled, transform=train_tfms)
train_ds = train_dataset.dataset()

val_dataset = PixWiseDataset(val_data, transform=test_tfms)
val_ds = val_dataset.dataset()

batch_size = 8
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)

trainer = Trainer(train_dl, val_dl, model, 12, opt, loss_fn, device)

print('Training Beginning\n')
trainer.fit()

print('\nTraining Complete')
torch.save(model.state_dict(), './models/DeePixBiS/DeePixBiS_celeb_nuaa_130223.pth')

In [ ]:
torch.save(model.state_dict(), './models/DeePixBiS_celeb_nuaa_130223.pth')

# Test

In [ ]:
# Setting device on GPU if available, else CPU
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    print('Max memory reserved: ', round(torch.cuda.max_memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
test_data = pd.read_csv('/kaggle/input/face-anti-spoofing/test_data_celeb_nuaa_kaggle.csv').reset_index(drop=True)

In [ ]:
test_model = DeePixBiS()
test_model.load_state_dict(torch.load('./models/DeePixBiS_celeb_nuaa_130223.pth'))
test_model = test_model.to(device)

loss_fn = PixWiseBCELoss()

test_tfms = Compose([Resize([224, 224]),
                     ToTensor(),
                    #  Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                     Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_dataset = PixWiseDataset(test_data, transform=test_tfms)
test_ds = test_dataset.dataset()

batch_size = 8
test_dl = DataLoader(test_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)

test_acc = test_accuracy(test_model, test_dl, device)
test_los = test_loss(test_model, test_dl, loss_fn, device)

print(test_acc, test_los)